# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [37]:
# imports
from pathlib import Path
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

In [38]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
type(mapbox_token)

str

# Import Data

In [39]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [40]:
# Define Panel Visualization Functions
def housing_units():
    """Housing Units Per Year."""
    
    housing_units = sfo_data.groupby('year').mean()
    housing_units_plot = plt.figure()
    max = housing_units.max()['housing_units']
    min = housing_units.min()['housing_units']
    housing_units.plot.bar(ylim =(min-3000, max+3000),title="Avg Housing Units per year in San Francisco", figsize=(10,7))
    
    plt.close(housing_units_plot) 
    return pn.pane.Matplotlib(housing_units_plot)


def avg_gross_rent():
    """Average Gross Rent in San Francisco"""
    
    gross_rent = pd.DataFrame(sfo_data.groupby(['year']).mean()['gross_rent'])
    
    gross_rent_plot = gross_rent.hvplot(figsize=(8, 8), title ='Avg gross rent')
    

    return gross_rent_plot



def avg_sales_price():
    """Average Sales Price"""
    
    average_price = pd.DataFrame(sfo_data.groupby(['year']).mean()['sale_price_sqr_foot'])
    
    average_price_plot = average_price.hvplot(figsize=(8, 8),title='Avg sales price per year' )
    
    
    return average_price_plot


def avg_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    average_price_nh = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
    average_price_nh.reset_index(inplace=True)
    average_price_nh.rename(columns={"level_0": "year"}, inplace=True)
    
    average_price_nh_plot = average_price_nh.hvplot.line(
        x="year",
        y="sale_price_sqr_foot",
        xlabel= "Year",
        ylabel="Average Price PSFT",
        groupby="neighborhood",
    )
    
    return average_price_nh_plot

def top_ten_neighborhoods():
    """Top Ten Most Expensive Neighborhoods in San Francisco."""
    
    top_ten = sfo_data.groupby("neighborhood").mean()
    top_ten = top_ten.sort_values("sale_price_sqr_foot", ascending=False).head(10)
    top_ten = top_ten.reset_index()
    top_ten
    
    
    top_ten_plot = top_ten.hvplot.bar(
    x="neighborhood",
    y="sale_price_sqr_foot",
    title="Top Ten  Most Expensive Neighborhoods in San Francisco",
    xlabel="Neighborhood",
    ylabel="Average Price PSFT",
    height=500,
    color="blue",
    rot=45
)
    
    return top_ten_plot


def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    top_ten = sfo_data.groupby("neighborhood").mean()
    top_ten = top_ten.sort_values("sale_price_sqr_foot", ascending=False).head(10)
    top_ten = top_ten.reset_index()
    top_ten
   
    parallel_coordinates_plot = px.parallel_coordinates(
    top_ten,
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    title='Average House Value in Neighborhood',
    labels={
        'neighborhood': "Neighborhood", 
        'sale_price_sqr_foot':'Sales Price/Square Foot',
        'housing_units':'Housing Units',
        'gross_rent':'Gross Rent',
    },
)
    
    return parallel_coordinates_plot



def parallel_categories():
    """Parallel Categories Plot."""
    
    top_ten = sfo_data.groupby("neighborhood").mean()
    top_ten = top_ten.sort_values("sale_price_sqr_foot", ascending=False).head(10)
    top_ten = top_ten.reset_index()
    top_ten
    
    parallel_categories_plot = px.parallel_categories(
    top_ten,
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    title='Average House Value in Neighborhood',
    labels={
        'neighborhood': "Neighborhood", 
        'sale_price_sqr_foot':'Sales Price/Square Foot',
        'housing_units':'Housing Units',
        'gross_rent':'Gross Rent',
    },
) 
       
    return parallel_categories_plot


def neighborhood_map():
    """Neighborhood Map"""
    
    avg_neighborhood = sfo_data.groupby("neighborhood").mean()
    avg_neighborhood = avg_neighborhood.reset_index()
    
    top_ten = sfo_data.groupby("neighborhood").mean()
    top_ten = top_ten.sort_values("sale_price_sqr_foot", ascending=False).head(10)
    top_ten = top_ten.reset_index()
    
    values_and_locations = pd.concat([
                                neighborhood_locations,
                                avg_neighborhood['sale_price_sqr_foot'],
                                avg_neighborhood['housing_units'],
                                avg_neighborhood ['gross_rent']
                               ], axis=1).dropna()
    
    px.set_mapbox_access_token(mapbox_token)
    
    map = px.scatter_mapbox(
        values_and_locations,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    color_continuous_scale=px.colors.cyclical.IceFire,
    size_max=30,
    zoom=10,
    width=1000,
    hover_name="Neighborhood",
    title="Average Price Psft and Average Gross Rent in San Francisco",
)
    
    plotly_panel = pn.pane.Plotly(map)
    plotly_panel._updates = True
    return plotly_panel



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [41]:
title = pn.pane.Markdown(
    """
# Analysis of San Francisco Real Estate from 2010 to 2016
""",
    width=800,
)

map = pn.pane.Markdown(
    """
Visual analysis of the San Francisco Real Estate
"""
)

tabs = pn.Tabs(
    ("Map", pn.Column(map, neighborhood_map())),
    ("Market Trend", pn.Row( avg_gross_rent(), avg_sales_price())),
    ("Neighborhood Trend", pn.Column(avg_price_by_neighborhood(), top_ten_neighborhoods())),
    (
        "Parallel Plots Analysis",
        pn.Column(parallel_coordinates(), parallel_categories(), width=1000),
    ),
)
panel = pn.Column(pn.Row(title), tabs, width=1000)

## Serve the Panel Dashboard

In [42]:
panel.servable()

Column(width=1000)
    [0] Row
        [0] Markdown(str, width=800)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Plotly(Figure)
        [1] Row
            [0] HoloViews(Curve)
            [1] HoloViews(Curve)
        [2] Column
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
            [1] HoloViews(Bars)
        [3] Column(width=1000)
            [0] Plotly(Figure)
            [1] Plotly(Figure)